In [2]:
import pandas as pd
import yaml
from sqlalchemy import create_engine
import polars

In [26]:
with open("../config/main.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

with open("../config/keys.yaml", "r") as f:
    keys = yaml.load(f, Loader=yaml.FullLoader)

import psycopg2

psy_engine = psycopg2.connect(
    database=config['database']['name'],
    user=config['database']['user'],
    password=keys['postgres'],
    host=config['database']['host'],
    port='5432'
)

cur = psy_engine.cursor()

In [4]:
# load in csv for attributes
df = pd.read_csv("/home/ubuntu/work/attributes/final.csv").iloc[:, 1:]

In [ ]:
# df2 = polars.read_csv("/home/ubuntu/work/attributes/final.csv")

In [12]:
df.dtypes

Unnamed: 0                    int64
corpusid                      int64
externalids                  object
url                          object
title                        object
authors                      object
venue                        object
publicationvenueid           object
year                        float64
referencecount                int64
citationcount                 int64
influentialcitationcount      int64
isopenaccess                   bool
s2fieldsofstudy              object
publicationtypes             object
publicationdate              object
journal                      object
updated                      object
id                            int64
dtype: object

In [5]:
df = df.drop("Unnamed: 0", axis = 1)

In [21]:
df = df.drop_duplicates(subset = 'corpusid')
index_col = df.pop('corpusid')

In [23]:
sql = create_engine(f'postgresql://postgres:{keys["postgres"]}@{config["database"]["host"]}:5432/postgres')

In [24]:
df.to_sql('attributes', con=sql, if_exists='replace', index=index_col, chunksize = 10000)

72817

In [28]:
test = cur.execute("SELECT * FROM attributes;")

In [29]:
cur.fetchone()

('{"ACL": null, "DBLP": null, "ArXiv": null, "MAG": "2889324973", "CorpusId": "52273203", "PubMed": "30272009", "DOI": "10.1038/s42003-018-0133-4", "PubMedCentral": "6123708"}',
 'https://www.semanticscholar.org/paper/39063bcddbd9decd4c8f5c5d3e8392280ab046ad',
 'Large conformational changes of a highly dynamic pre-protein binding domain in SecA',
 '{"{\\"authorId\\": \\"51285510\\", \\"name\\": \\"Isabel Ernst\\"}","{\\"authorId\\": \\"10784515\\", \\"name\\": \\"Maximilian Haase\\"}","{\\"authorId\\": \\"47832514\\", \\"name\\": \\"S. Ernst\\"}","{\\"authorId\\": \\"46262768\\", \\"name\\": \\"Shuguang Yuan\\"}","{\\"authorId\\": \\"46371491\\", \\"name\\": \\"A. Kuhn\\"}","{\\"authorId\\": \\"6835497\\", \\"name\\": \\"S. Leptihn\\"}"}',
 'Communications Biology',
 '069e05e7-ca35-41d0-a8c7-bdc9ec6a82af',
 2018.0,
 28,
 13,
 0,
 True,
 '{"{\\"category\\": \\"Biology\\", \\"source\\": \\"s2-fos-model\\"}","{\\"category\\": \\"Chemistry\\", \\"source\\": \\"s2-fos-model\\"}","{\\"catego

In [30]:
psy_engine.commit()
cur.close()
psy_engine.close()